In [ ]:
#this will have extended implementation of basic Policy gradient function and here we will have additional
#network function neural network as a critic to judge the actor's actions and improve its odds of balancing the CartPole
#this is insipred from open AI article on PPO and below research paper:
https://arxiv.org/pdf/1707.06347


In [ ]:
#only thing that is special in PPo is that critic sends an advantage function to the actor and then
based on the ratio between new policy and old policy ( this is the advantage fxn) we clip the update to a certan range and then run the update
this helps in avoiding too much drastic change in network and kind of adds a slow learning ensuring it learns better in time 